# Preamble

In [12]:
%matplotlib inline
from my_units import *
from load_functions import *
from yellin_functions import *

In [13]:
dir_home = '/Users/kvantilburg/Dropbox/projects/LuminousBasin/LuminousBasin/luminous-basin/'
dir_events = dir_home+'mocks/event_lists/'
dir_source = dir_home+'data/likelihood_yellin_data/products_80610202001_orbit1_chu12_spatialARF/'
dir_production = dir_home+'data/production/'

# Parameters

In [14]:
i_mock = '6' # mock file number
m = 12 # axion mass [keV]
sigma_E = 0.166 # energy resolution [keV]

In [15]:
# time intervals [seconds]
good_time_ints = np.asarray([
    [3.37603341e+08, 3.376033795e+08],
    [3.376036305e+08, 3.3760522972e+08]
])
exposure = ((good_time_ints[0,1]-good_time_ints[0,0]) + (good_time_ints[1,1]-good_time_ints[1,0]))
livetime = np.asarray([1501.16599845754, 1481.86081041239])/exposure # effective fractional livetime of A and B
livetime 

array([0.91661945, 0.9048316 ])

In [16]:
# time bins
N_bins_t = 10
bins_t = np.linspace(good_time_ints[0][0], good_time_ints[-1][1], N_bins_t+1)
# energy bins
width_E = 0.04 # energy bin width is 40 keV
bins_E = np.arange(1.6,200.01,width_E)

In [17]:
# fiducial solar position [degrees]
alpha0 = 170.66855149
delta0 = 4.02092024
# error on solar

In [18]:
list_file_events = [dir_events+file for file in listdir(dir_events) if file[0:len(i_mock)+1]==i_mock+'_']
file_box_centers = dir_source+'box_centers.txt'
list_file_arf = [dir_source+'arfs/'+file for file in listdir(dir_source+'arfs/')]

# Load data

In [19]:
df_data = load_data(m,sigma_E,good_time_ints,livetime,bins_t,bins_E,list_file_events,file_box_centers,list_file_arf)

339it [00:35,  9.65it/s]


In [20]:
df_data

,detector,idx_t,idx_E,i1,i2,t,E,ra,dec,exp,arf,Omega,counts
0,A,0,102,-6,-6,3.376033e+08,5.68,170.889,3.75981,0.000000,97.389603,1.0,0
1,A,0,102,-6,-5,3.376033e+08,5.68,170.881,3.77425,0.000000,71.114639,1.0,0
2,A,0,102,-6,-4,3.376033e+08,5.68,170.872,3.78868,0.000000,117.963196,1.0,0
3,A,0,102,-6,-3,3.376033e+08,5.68,170.864,3.80311,0.000000,116.242241,1.0,0
4,A,0,102,-6,-2,3.376033e+08,5.68,170.856,3.81755,0.000000,203.614990,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57455,B,9,118,6,2,3.376050e+08,6.32,171.009,3.97187,170.897354,121.035553,1.0,0
57456,B,9,118,6,3,3.376050e+08,6.32,171.001,3.98630,170.897354,99.170128,1.0,0
57457,B,9,118,6,4,3.376050e+08,6.32,170.992,4.00073,170.897354,71.558678,1.0,0
57458,B,9,118,6,5,3.376050e+08,6.32,170.984,4.01517,170.897354,70.092308,1.0,0


In [21]:
t = df_data['t'].to_numpy()
E = df_data['E'].to_numpy()
ra = df_data['ra'].to_numpy()
dec = df_data['dec'].to_numpy()
eps = df_data['exp'].to_numpy() * df_data['arf'].to_numpy() * df_data['Omega'].to_numpy()
counts = df_data['counts'].to_numpy()

In [22]:
np.sum(counts)

10561